from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import re
from datasets import Dataset
import numpy as np
import evaluate
from transformers import TrainingArguments, Trainer
from functools import partial
from sklearn.metrics import accuracy_score




file1_path="/mnt/data-poseidon/sumire/thesis/2-1/en-ja/Llama-2-70b-instruct-v2-usas-zs-p1-nsplit-ja-2-1/prompt+source.txt"
file2_path = "/mnt/data-poseidon/sumire/thesis/2-1/en-ja/Llama-2-70b-instruct-v2-usas-zs-p1-nsplit-ja-2-1/source.txt"
label_path = "/mnt/data-poseidon/sumire/thesis/2-1/en-ja/Llama-2-70b-instruct-v2-usas-zs-p1-nsplit-ja-2-1/comet_binary.txt"


def concatenate_lines(file1_path, file2_path):
    # Read the lines from both files
    with open(file1_path, 'r') as file1:
        lines1 = file1.readlines()
    with open(file2_path, 'r') as file2:
        lines2 = file2.readlines()

    # Concatenate lines from the two files to create instances
    instances =  [ line1.strip() + line2.strip() for line1, line2 in zip(lines1, lines2)]

    return instances
# Load and concatenate the data
instances = concatenate_lines(file1_path, file2_path)

# Label load
all_labels = []
with open (label_path, 'r')as file:
    for line in file:
        score = int(line.strip())
        all_labels.append(score)

# Define the index where you want to split the dataset
split_index = 100  # Replace this with your desired index

# Split the custom_dataset into training and validation
train_dataset = Dataset.from_dict({'text': instances[:split_index], 'label':all_labels[:split_index]})
validation_dataset = Dataset.from_dict({"text": instances[split_index:], 'label':all_labels[split_index:]})

train_dataset = train_dataset.shuffle(seed=42)
validation_dataset = validation_dataset.shuffle(seed=42)
# Access the training and validation datasets
print("Training dataset:")
print(len(train_dataset))

#print("Validation dataset:")
print(len(validation_dataset))

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

train_tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
val_tokenized_datasets = validation_dataset.map(tokenize_function, batched=True)

small_train_dataset = train_tokenized_datasets.select(range(10)) ##########
small_eval_dataset = val_tokenized_datasets.select(range(10)) #######

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=5)


training_args = TrainingArguments(
    output_dir="test_trainer",
    do_eval= True,
    do_predict= True,
    do_train=True, 
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
    )

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred, metric):
    _ = eval_pred 
    #logits, labels = eval_pred
    predictions, labels
    print(eval_pred)
    #print (logits, labels)
    #predictions = np.argmax(logits, axis=-1)
    
    #accuracy = accuracy_score(y_true = labels, y_pred=predictions)
    
    #accuracy = metric.compute(predictions=predictions, references=labels)

    result = {"accuracy": 2}
    
    return result
    

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=small_train_dataset,
    eval_dataset=small_eval_dataset,
    compute_metrics=partial(compute_metrics,metric)
)

trainer.train()

model.eval()
trainer.evaluate()
#preds, label_ids, metrics = trainer.predict(small_eval_dataset)##############put test data


In [35]:
path = "/mnt/data-poseidon/sumire/thesis/running/ted/eval_mt/train-val/testing_Llama-2-70b-instruct-v2-usas-zs-p1-nsplit-ja-1-1/prompt+source.txt"
# Read your input file
with open(path, 'r') as file:
    text = file.read()

# Use regular expressions to find and extract the sentences after "Statement A"
matches = re.findall(r'Given context:\n(.*?)\nTranslate English to Japanese:', text, re.DOTALL)

# Join the matched sentences within each "Statement A" with commas
joined_sentences = [''.join(match.strip().split('\n')) for match in matches]

# Join the matched sentences into a single string
extracted_text = '\n'.join(joined_sentences)

# Save the extracted text into a new file
with open('extracted.txt', 'w') as output_file:
    output_file.write(extracted_text)


In [40]:
from datasets import Dataset
import re
file1_path="/home/sumire/thesis/LLM_Contextual_Prompt_MT/main/extracted.txt"
file2_path = "/mnt/data-poseidon/sumire/thesis/running/ted/eval_mt/train-val/testing_Llama-2-70b-instruct-v2-usas-zs-p1-nsplit-ja-1-1/source.txt"

def concatenate_lines(file1_path, file2_path):
    # Read the lines from both files
    with open(file1_path, 'r') as file1:
        lines1 = file1.readlines()
    with open(file2_path, 'r') as file2:
        lines2 = file2.readlines()

    # Concatenate lines from the two files to create instances
    instances =  [ line1.strip() + line2.strip() for line1, line2 in zip(lines1, lines2)]

    return instances

# Load and concatenate the data
instances = concatenate_lines(file1_path, file2_path)

# Access the dataset
print(instances[200])


Context sentence is here!?yeeees!It's not like they're mutually exclusive or anything.


In [44]:
# Define the index where you want to split the dataset
split_index = 100  # Replace this with your desired index

# Split the custom_dataset into training and validation
train_dataset = Dataset.from_dict({'text': instances[:split_index]})
print (train_dataset)
validation_dataset = Dataset.from_dict({"text": instances[split_index:]})
dataset = {
    'train': train_dataset,
    'val':  validation_dataset
}

# Access the training and validation datasets
print("Training dataset:")
print(dataset['train'])

#print("Validation dataset:")
print(dataset['val']['text'])

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = train_dataset.map(tokenize_function, batched=True)
print (tokenized_datasets)



Dataset({
    features: ['text'],
    num_rows: 100
})
Training dataset:
Dataset({
    features: ['text'],
    num_rows: 100
})
['Context sentence is here!?yeeees!When you do break it, I\'m going to give you a hundred bucks."', "Yaaaaaaaaaaaaaaaaaaaaaaaaaaa3221=Now, of course, this was a simple ploy on Dr. P's part  to get me to do the exercises I didn't want to do  before the prospect of being the richest five-year-old in the second floor ward,  but what he effectively did for me was reshape an awful daily occurrence  into a new and promising experience for me.", 'Context sentence is here!?yeeees!and his declaration of me as a strong and powerful little girl  shaped my own view of myself  as an inherently strong, powerful and athletic person well into the future.', 'Yaaaaaaaaaaaaaaaaaaaaaaaaaaa3221=This is an example of how adults in positions of power  can ignite the power of a child.', "Context sentence is here!?yeeees!But, in the previous instances of those thesaurus entries,  our 

  0%|          | 0/1 [00:00<?, ?ba/s]

Dataset({
    features: ['text', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 100
})


In [42]:
from datasets import load_dataset
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import TrainingArguments
import re
from datasets import Dataset

file1_path="/home/sumire/thesis/LLM_Contextual_Prompt_MT/main/extracted.txt"
file2_path = "/mnt/data-poseidon/sumire/thesis/running/ted/eval_mt/train-val/testing_Llama-2-70b-instruct-v2-usas-zs-p1-nsplit-ja-1-1/source.txt"
def concatenate_lines(file1_path, file2_path):
    # Read the lines from both files
    with open(file1_path, 'r') as file1:
        lines1 = file1.readlines()
    with open(file2_path, 'r') as file2:
        lines2 = file2.readlines()

    # Concatenate lines from the two files to create instances
    instances =  [ line1.strip() + line2.strip() for line1, line2 in zip(lines1, lines2)]

    return instances

# Load and concatenate the data
instances = concatenate_lines(file1_path, file2_path)

# Access the dataset
print(instances[200])

# Define the index where you want to split the dataset
split_index = 100  # Replace this with your desired index

# Split the custom_dataset into training and validation
train_dataset = Dataset.from_dict({'text': instances[:split_index]})
print (train_dataset)
validation_dataset = Dataset.from_dict({"text": instances[split_index-1:len(instances)]})

dataset = Dataset({
    'train': train_dataset,
    'val':  validation_dataset
})

Context sentence is here!?yeeees!It's not like they're mutually exclusive or anything.
Dataset({
    features: ['text'],
    num_rows: 100
})


ArrowInvalid: Column 1 named val expected length 100 but got length 275

In [1]:
from datasets import load_dataset


Generating train split:   0%|          | 0/650000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/50000 [00:00<?, ? examples/s]

Dataset yelp_review_full downloaded and prepared to /home/sumire/.cache/huggingface/datasets/yelp_review_full/yelp_review_full/1.0.0/e8e18e19d7be9e75642fc66b198abadb116f73599ec89a69ba5dd8d1e57ba0bf. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

{'label': 0,
 'text': 'My expectations for McDonalds are t rarely high. But for one to still fail so spectacularly...that takes something special!\\nThe cashier took my friends\'s order, then promptly ignored me. I had to force myself in front of a cashier who opened his register to wait on the person BEHIND me. I waited over five minutes for a gigantic order that included precisely one kid\'s meal. After watching two people who ordered after me be handed their food, I asked where mine was. The manager started yelling at the cashiers for \\"serving off their orders\\" when they didn\'t have their food. But neither cashier was anywhere near those controls, and the manager was the one serving food to customers and clearing the boards.\\nThe manager was rude when giving me my order. She didn\'t make sure that I had everything ON MY RECEIPT, and never even had the decency to apologize that I felt I was getting poor service.\\nI\'ve eaten at various McDonalds restaurants for over 30 years. 

In [2]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)

  0%|          | 0/650 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [3]:

small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

In [7]:
! pip install accelerate -U

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [8]:

from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="./running")

ImportError: Using the `Trainer` with `PyTorch` requires `accelerate>=0.20.1`: Please run `pip install transformers[torch]` or `pip install accelerate -U`

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [2]:
a = range(1,5+1)
print (a)

for i in a:
    print(i)

range(1, 6)
1
2
3
4
5


In [1]:
from extractive import ExtractiveSummarizer
import torch

#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_checkpoint= "/mnt/data-poseidon/sumire/repos/transformersum/ckpt/epoch=3.ckpt"
model = ExtractiveSummarizer.load_from_checkpoint(model_checkpoint)

#model.to(device)  
text_to_summarize = "And keep it clean!Mr. Chandler!My friends, you are all familiar with Beaugard.This is perhaps his masterpiece."
input_sentences = ["And keep it clean!", "Mr. Chandler!", "My friends, you are all familiar with Beaugard.", "This is perhaps his masterpiece."]
summary = model.predict_sentences(input_sentences, raw_scores=False, num_summary_sentences=1)
print (summary)

ModuleNotFoundError: No module named 'extractive'